In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer, transpile, visualization, circuit
from qiskit.aqua import QuantumInstance
import math

In [ ]:
def register_cnot(circuit,x,t,type,n_ctrl):
    if type=='0':
        for i in range(0,n_ctrl):
            circuit.x(x[i])
        circuit.mcx(x,t)
        for i in range(0,n_ctrl):
            circuit.x(x[i])
    else:
        circuit.mcx(x,t)

In [ ]:
def round(circuit,u,v,ca,s,r,cb,cc,m,f,k,l,n):
    register_cnot(circuit,k,f,'0',l) #the qubit is toggled if k is 0, so we add another x-gate
    circuit.x(f)
    register_cnot(circuit,v,f,'0',n)
    #circuit.x(f)
    circuit.x(f)
    ECC_library.incremental_gate_controlled(circuit,f[0:1]+k[0:l],v[0:l],l+1)
    circuit.x(f)
    #creating sub_circuit in order to apply the control to the block
    ECC_library.controlled_block(circuit,f,u,v,ca,s,r,cb,cc,m,n)
    

In [ ]:
a = 8
p = 11
n = 4
R = (2**n)%p
s_a=bin(int(a))[2:].zfill(n)
s_p=bin(int(p))[2:].zfill(n)
s_p=s_p[::-1]
s_p1=s_p+'0'
l = math.ceil(math.log(n,2))


u = QuantumRegister(n,'u')
v = QuantumRegister(n,'v')
ca = QuantumRegister(1,'ca')
s = QuantumRegister(n,'s')
r = QuantumRegister(n,'r')
cb = QuantumRegister(1,'cb')
cc = QuantumRegister(1,'cc')
m = QuantumRegister(2*n,'m')
f = QuantumRegister(1,'f')
k = QuantumRegister(l,'k')
clas_u = ClassicalRegister(n)
clas_v = ClassicalRegister(n)
clas_s = ClassicalRegister(n)
clas_r = ClassicalRegister(n)
clas_k = ClassicalRegister(l)
circuit = QuantumCircuit(u,v,ca,s,r,cb,cc,m,f,k,clas_u,clas_v,clas_s,clas_r,clas_k)
for i in range(0,n):
    if s_a[i] == '1':
        circuit.x(v[n-1-i])
circuit.x(s[0])
circuit.barrier(u,v,ca,s,r,cb,cc,m,f,k)
circuit.x(f)
for i in range(0,2*n):
    round(circuit,u,v,ca,s,r,cb,cc,m[i:i+1],f,k,l,n)
"""now we have to copy r into another register"""

"""now the modular doublings (k times)"""

"""now the sign flip"""


for i in range(0,n):
    circuit.measure(u[i],clas_u[i])
for i in range(0,n):
    circuit.measure(v[i],clas_v[i])
for i in range(0,n):
    circuit.measure(s[i],clas_s[i])
for i in range(0,n):
    circuit.measure(r[i],clas_r[i])
for i in range(0,l):
    circuit.measure(k[i],clas_k[i])
#circuit.draw()

In [ ]:
IBMQ.load_account();
provider = IBMQ.get_provider('ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')
simulation = execute(circuit, backend=backend ,shots=1)
sim_result = simulation.result()
out=list(sim_result.get_counts().keys())[0]
output=out.split(' ')
output_u = output[4]
output_v = output[3]
output_s = output[2]
output_r = output[1]
output_k = output[0]
res_u = int(output_u, 2)
res_v = int(output_v, 2)
res_s = int(output_s, 2)
res_r = int(output_r, 2)
res_k = int(output_k, 2)
print('res_u:',res_u,'res_v:',res_v,'res_s:',res_s,'res_r:',res_r,'res_k:',res_k)